In [1]:
import agents
from dotenv import load_dotenv
# from openai import OpenAI
# openai_client = OpenAI()
load_dotenv()



True

In [2]:
agents.__version__

'0.3.3'

In [3]:
import requests


# Create the function
def fetch_url(url):
    reader_url_prefix = 'https://r.jina.ai/'
    request_url = reader_url_prefix + url
    response = requests.get(request_url)
    return response.content.decode('utf8')

In [4]:
# fetch_url('https://datatalks.club')

In [5]:
import requests
from requests.exceptions import RequestException
from typing import Optional


# Make the function with docstring, and type -> Easily parsed by the agent 
def fetch_url(url: str) -> Optional[str]:
    """
    Fetch the textual content of a webpage.

    Args:
        url (str): The target URL to fetch content from.

    Returns:
        Optional[str]: The decoded HTML/text content of the fetched page if successful,
        or None if an error occurred.

    Raises:
        ValueError: If the provided URL is empty or invalid.
    """
    if not url or not isinstance(url, str):
        raise ValueError("The 'url' parameter must be a non-empty string.")

    jina_reader_base_url = "https://r.jina.ai/"
    jina_reader_url = jina_reader_base_url + url.lstrip("/")

    try:
        response = requests.get(jina_reader_url, timeout=10)
        response.raise_for_status()  # Raises HTTPError for bad status codes
        return response.content.decode("utf-8")
    except RequestException as e:
        # Catch all network-related errors (e.g., ConnectionError, Timeout, HTTPError)
        print(f"Error fetching URL '{jina_reader_url}': {e}")
        return None
    except UnicodeDecodeError:
        print(f"Error decoding response from '{jina_reader_url}'.")
        return None

In [6]:
from agents import Agent, function_tool

In [7]:
# build the agent 
# get the runner 

In [8]:
# Build the agent with the tool, system instruction and models
assitant_instructions = """
You're a helpful assistant that helps answer user questions.
"""

web_agent = Agent(
    name='web_agent',
    tools=[function_tool(fetch_url)],
    instructions=assitant_instructions,
    model='gpt-4o-mini'
)

In [9]:
from agents import Runner

# Start running ..     with user prompt
runner = Runner()

In [10]:
user_prompt = "Summarize the content of https://openai.github.io/openai-agents-python/"

results = await runner.run(web_agent, input=user_prompt)

In [11]:
results

RunResult(input='Summarize the content of https://openai.github.io/openai-agents-python/', new_items=[ToolCallItem(agent=Agent(name='web_agent', handoff_description=None, tools=[FunctionTool(name='fetch_url', description='Fetch the textual content of a webpage.', params_json_schema={'properties': {'url': {'description': 'The target URL to fetch content from.', 'title': 'Url', 'type': 'string'}}, 'required': ['url'], 'title': 'fetch_url_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7f8160989510>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)], mcp_servers=[], mcp_config={}, instructions="\nYou're a helpful assistant that helps answer user questions.\n", prompt=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, para

In [14]:
items = results.new_items

In [15]:
items

[ToolCallItem(agent=Agent(name='web_agent', handoff_description=None, tools=[FunctionTool(name='fetch_url', description='Fetch the textual content of a webpage.', params_json_schema={'properties': {'url': {'description': 'The target URL to fetch content from.', 'title': 'Url', 'type': 'string'}}, 'required': ['url'], 'title': 'fetch_url_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7f8160989510>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)], mcp_servers=[], mcp_config={}, instructions="\nYou're a helpful assistant that helps answer user questions.\n", prompt=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None

In [16]:
for item in items:
    print(item.agent.tools)

[FunctionTool(name='fetch_url', description='Fetch the textual content of a webpage.', params_json_schema={'properties': {'url': {'description': 'The target URL to fetch content from.', 'title': 'Url', 'type': 'string'}}, 'required': ['url'], 'title': 'fetch_url_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7f8160989510>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)]
[FunctionTool(name='fetch_url', description='Fetch the textual content of a webpage.', params_json_schema={'properties': {'url': {'description': 'The target URL to fetch content from.', 'title': 'Url', 'type': 'string'}}, 'required': ['url'], 'title': 'fetch_url_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7f8160989510>, strict_json_schema=True, is_en

In [17]:
from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import OpenAIAgentsSDKRunner

chat_interface = IPythonChatInterface()

runner = OpenAIAgentsSDKRunner(
    chat_interface=chat_interface,
    agent=web_agent
)

In [18]:
# await runner.run();

### Youtube Video

In [19]:
# Youtube video


from youtube_transcript_api import YouTubeTranscriptApi

def format_timestamp(seconds: float) -> str:
    """Convert seconds to H:MM:SS if > 1 hour, else M:SS"""
    total_seconds = int(seconds)
    hours, remainder = divmod(total_seconds, 3600)
    minutes, secs = divmod(remainder, 60)

    if hours > 0:
        return f"{hours}:{minutes:02}:{secs:02}"
    else:
        return f"{minutes}:{secs:02}"


def make_subtitles(transcript) -> str:
    lines = []

    for entry in transcript:
        ts = format_timestamp(entry.start)
        text = entry.text.replace('\n', ' ')
        lines.append(ts + ' ' + text)

    return '\n'.join(lines)


def fetch_transcript_raw(video_id):
    ytt_api = YouTubeTranscriptApi()
    transcript = ytt_api.fetch(video_id)
    return transcript


def fetch_transcript_text(video_id):
    transcript = fetch_transcript_raw(video_id)
    subtitles = make_subtitles(transcript)
    return subtitles

In [20]:
def fetch_youtube_transcript(video_id: str) -> str:
    """
    Fetches the transcript of a YouTube video and converts it into a subtitle-formatted string.

    

    Example:
    0:00 Hey everyone, welcome to our event. This
    0:02 event is brought to you by data talks
    0:03 club which is a community of people who
    0:05 love data. We have weekly events today.
    0:08 Uh this is one of such events. Um if you
    0:11 want to find out more about the events
    0:13 we have, there is a link in the
    0:14 description. Um so click on that link,
    0:16 check it out right now. We actually have
    0:19 quite a few events in our pipeline, but
    0:21 we need to put them on the website. Uh
    0:24 but keep a

    Args:
        video_id (str): The unique YouTube video ID.

    Returns:
        str: The subtitles generated from the video's transcript.
    """
    return fetch_transcript_text(video_id)

In [21]:
subtitles = fetch_youtube_transcript('vK_SxyqIfwk')
print(subtitles[:500])

0:00 Hey everyone, welcome to our event. This
0:02 event is brought to you by data talks
0:03 club which is a community of people who
0:05 love data. We have weekly events today.
0:08 Uh this is one of such events. Um if you
0:11 want to find out more about the events
0:13 we have, there is a link in the
0:14 description. Um so click on that link,
0:16 check it out right now. We actually have
0:19 quite a few events in our pipeline, but
0:21 we need to put them on the website. Uh
0:24 but keep a


In [22]:
summary_instructions = """
You're a helpful assistant that helps answer user questions
about YouTube videos
"""

In [23]:
from agents import Runner

# Start running ..     with user prompt
runner = Runner()

In [24]:
youtube_agent = Agent(
    name='youtube_agent',
    instructions=summary_instructions,
    tools=[function_tool(fetch_youtube_transcript)],
    model='gpt-4o-mini'
)

In [31]:
user_prompt = """What is this video about https://www.youtube.com/watch?v=vK_SxyqIfwk$t=172s
    Output the Example like this:
    0:00 Hey everyone, welcome to our event. This
    0:02 event is brought to you by data talks
    0:03 club which is a community of people who
    0:05 love data. We have weekly events today.
    0:08 Uh this is one of such events. Um if you
    0:11 want to find out more about the events
    0:13 we have, there is a link in the
    0:14 description. Um so click on that link,
    0:16 check it out right now. We actually have
    0:19 quite a few events in our pipeline, but
    0:21 we need to put them on the website. Uh
    0:24 but keep a
"""

In [32]:
runner = Runner()
results = await runner.run(youtube_agent, input=user_prompt)

In [33]:
print(type(results))
print(results)

<class 'agents.result.RunResult'>
RunResult:
- Last agent: Agent(name="youtube_agent", ...)
- Final output (str):
    Here's a summary of the video, formatted as requested:
    
    ```
    0:00 Hey everyone, welcome to our event. This
    0:02 event is brought to you by data talks
    0:03 club which is a community of people who
    0:05 love data. We have weekly events today.
    0:08 Uh this is one of such events. Um if you
    0:11 want to find out more about the events
    0:13 we have, there is a link in the
    0:14 description. Um so click on that link,
    0:16 check it out right now. We actually have
    0:19 quite a few events in our pipeline, but
    0:21 we need to put them on the website. Uh
    0:24 but keep an eye on it anyways. Um so we
    0:27 will put um them. you'll see them. And
    0:30 then don't forget to subscribe to our
    0:32 YouTube channel. So this is the most um
    0:35 reliable way of getting notified when
    0:37 our stream starts. And last but not


In [22]:
from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import OpenAIAgentsSDKRunner

chat_interface = IPythonChatInterface()

In [23]:
runner = OpenAIAgentsSDKRunner(
    chat_interface=chat_interface,
    agent=youtube_agent
)

In [24]:
await runner.run();

You: What is this video about https://www.youtube.com/watch?v=vK_SxyqIfwk$t=172s


You: What is this video about https://www.youtube.com/watch?v=vK_SxyqIfwk$t=172s Give the example like this       Example:     0:00 Hey everyone, welcome to our event. This     0:02 event is brought to you by data talks     0:03 club which is a community of people who     0:05 love data. We have weekly events today.     0:08 Uh this is one of such events. Um if you     0:11 want to find out more about the events     0:13 we have, there is a link in the     0:14 description. Um so click on that link,     0:16 check it out right now. We actually have     0:19 quite a few events in our pipeline, but     0:21 we need to put them on the website. Uh     0:24 but keep a


You: stop


Chat ended.
